# 引例

In [1]:
import qlib
import pandas as pd
from qlib.constant import REG_CN
from qlib.utils import exists_qlib_data, init_instance_by_config
from qlib.workflow import R # 实验记录管理器
from qlib.workflow.record_temp import SignalRecord, SigAnaRecord, PortAnaRecord
from qlib.utils import flatten_dict
from qlib.data import D # 基础行情数据服务对象

provider_uri = "~/.qlib/qlib_data/cn_data"
qlib.init(provider_uri=provider_uri, region=REG_CN)

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
[3149:MainThread](2025-12-18 09:45:02,252) INFO - qlib.Initialization - [config.py:452] - default_conf: client.
[3149:MainThread](2025-12-18 09:45:02,256) INFO - qlib.Initialization - [__init__.py:75] - qlib successfully initialized based on client settings.
[3149:MainThread](2025-12-18 09:45:02,258) INFO - qlib.Initialization - [__init__.py:77] - data_path={'__DEFAULT_FREQ': PosixPath('/home/nbcctwya/.qlib/qlib_data/cn_data')}


In [2]:
stockpool = D.instruments(market='csi100')

benchmark = "SH000300" # 基准：沪深300指数

# 参数配置
# 数据处理器参数配置
data_handler_config = {
    "start_time": "2008-01-01",
    "end_time": "2020-08-31",
    "fit_start_time": "2008-01-01",
    "fit_end_time": "2014-12-31",
    "instruments": stockpool, ### 记住是instruments而不是instrument
}

# 任务参数配置
task = {
    # 机器学习模型参数配置
    "model": {
        # 模型类
        "class": "LGBModel",
        # 模型类所在模块
        "module_path": "qlib.contrib.model.gbdt",
        # 模型类超参数配置, 未写的采用默认值, 参数将传给模型类
        "kwargs": {
            "loss": "mse",
            "colsample_bytree": 0.8879,
            "learning_rate": 0.0421,
            "subsample": 0.8789,
            "lambda_l1": 205.6999,
            "lambda_l2": 580.9768,
            "max_depth": 8,
            "num_leaves": 210,
            "num_threads": 20,
            "early_stopping_rounds": 50, # 训练迭代提前停止条件
            "num_boost_round": 1000, # 最大训练迭代次数
        },
    },
    "dataset":{ # 因子数据集参数配置
        # 数据集类, 是Dataset with Data(H)andler的缩写
        "class": "DatasetH",
        # 数据集所在的模块
        "module_path": "qlib.data.dataset",
        # 数据集类的参数配置
        "kwargs": {
            "handler": {
                "class": "Alpha360",# 数据处理类,继承自DataHandlerLP
                "module_path": "qlib.contrib.data.handler",
                "kwargs": data_handler_config,
            },
            "segments":{ # 数据集时段划分
                "train": ("2008-01-01", "2014-12-31"),
                "valid": ("2014-12-31", "2016-12-31"),
                "test": ("2017-01-01", "2020-08-01"),
            },
        },
    },
}

# 实例化因子库数据集，从基础行情数据计算出的包括所有特征（因子）值和标签值的数据集
dataset = init_instance_by_config(task["dataset"])

[3149:MainThread](2025-12-18 09:45:12,769) INFO - qlib.timer - [log.py:127] - Time cost: 8.406s | Loading data Done
[3149:MainThread](2025-12-18 09:47:00,299) INFO - qlib.timer - [log.py:127] - Time cost: 107.280s | ProcessInf Done
/home/nbcctwya/anaconda3/envs/qlib_experiment/lib/python3.9/site-packages/qlib/data/dataset/processor.py:241: RuntimeWarning: Mean of empty slice
  self.mean_train = np.nanmean(df[cols].values, axis=0)
/home/nbcctwya/anaconda3/envs/qlib_experiment/lib/python3.9/site-packages/numpy/lib/_nanfunctions_impl.py:2035: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
[3149:MainThread](2025-12-18 09:47:01,088) INFO - qlib.timer - [log.py:127] - Time cost: 0.788s | ZScoreNorm Done
[3149:MainThread](2025-12-18 09:47:01,791) INFO - qlib.timer - [log.py:127] - Time cost: 0.703s | Fillna Done
[3149:MainThread](2025-12-18 09:47:01,923) INFO - qlib.timer - [log.py:127] - Time cost: 0.056s | DropnaLabel Done
[3